In [36]:
from dotenv import load_dotenv
import os
load_dotenv()



True

In [37]:
import weaviate
from weaviate.classes.init import Auth

# Best practice: store your credentials in environment variables
weaviate_url = os.getenv('WEAVIATE_URL')
weaviate_api_key = os.getenv('weaviate_api_key')

# Connect to Weaviate Cloud
client = weaviate.connect_to_weaviate_cloud(
    cluster_url=weaviate_url,
    auth_credentials=Auth.api_key(weaviate_api_key),
)

print(client.is_ready())

True


In [38]:
from langchain_huggingface import HuggingFaceEmbeddings
from dotenv import load_dotenv

load_dotenv()


embedding_model_id = "sentence-transformers/all-MiniLM-L6-v2"
embedding  = HuggingFaceEmbeddings(model_name = embedding_model_id)

In [39]:
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
loader = TextLoader("state_of_union.txt", encoding='utf-8')
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
docs = text_splitter.split_documents(documents)

In [40]:
from langchain_weaviate.vectorstores import WeaviateVectorStore
vector_db = WeaviateVectorStore.from_documents(documents=docs, embedding=embedding, client=client, by_text=False,
                                               kwargs={'alpha': 0.5})
vectorstore_retriever = vector_db.as_retriever(search_kwargs={'k':3})

In [43]:
from langchain.retrievers import BM25Retriever, EnsembleRetriever
keyword_retriever = BM25Retriever.from_documents(documents=docs)
keyword_retriever.k = 3

In [44]:
retriver = EnsembleRetriever(retrievers=[vectorstore_retriever, keyword_retriever], weights=[0.3,0.7])

In [ ]:
vector_db.similarity_search('what is content?', k=3, alpha=0.3) ## hybrid searc

[Document(metadata={'source': 'state_of_union.txt'}, page_content='Putin’s latest attack on Ukraine was premeditated and unprovoked. \n\nHe rejected repeated efforts at diplomacy. \n\nHe thought the West and NATO wouldn’t respond. And he thought he could divide us at home. Putin was wrong. We were ready.  Here is what we did.   \n\nWe prepared extensively and carefully. \n\nWe spent months building a coalition of other freedom-loving nations from Europe and the Americas to Asia and Africa to confront Putin. \n\nI spent countless hours unifying our European allies. We shared with the world in advance what we knew Putin was planning and precisely how he would try to falsely justify his aggression.  \n\nWe countered Russia’s lies with truth.   \n\nAnd now that he has acted the free world is holding him accountable. \n\nAlong with twenty-seven members of the European Union including France, Germany, Italy, as well as countries like the United Kingdom, Canada, Japan, Korea, Australia, New Z

In [45]:
from langchain.prompts import ChatPromptTemplate

template = 'You are an assistant for question-answering task.Use the following pieces of retrieved contect to answer the question.' \
            'If you dont know the answer just say that you dont know .Use ten sentences maximum and keep the answer concise' \
            'Question : {question}'\
            'Context : {context}'\
            'Answer ::'

prompt = ChatPromptTemplate.from_template(template=template)
print(prompt)


input_variables=['context', 'question'] input_types={} partial_variables={} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='You are an assistant for question-answering task.Use the following pieces of retrieved contect to answer the question.If you dont know the answer just say that you dont know .Use ten sentences maximum and keep the answer conciseQuestion : {question}Context : {context}Answer ::'), additional_kwargs={})]


In [46]:
from langchain_community.llms.huggingface_hub import HuggingFaceHub
from langchain_google_genai import ChatGoogleGenerativeAI

model = ChatGoogleGenerativeAI(model='gemini-2.5-flash-preview-04-17')

c:\Contents\Study Materials\KaggleData\Projects\GenAI\RAG_application_using_Gemini_and_Weviatedb\venv\Lib\site-packages\langchain_google_genai\chat_models.py:1022: PydanticDeprecatedSince211: Accessing the 'model_fields' attribute on the instance is deprecated. Instead, you should access this attribute from the model class. Deprecated in Pydantic V2.11 to be removed in V3.0.
  for field_name, field_info in self.model_fields.items():


In [47]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

rag_chain = ({'context' : retriver, 'question' : RunnablePassthrough()} |
             prompt | model | StrOutputParser())

In [48]:
rag_chain.invoke('What is the document about?')


"Based on the retrieved content, the document appears to be a State of the Union address. It reports on the current condition of the nation, stating that the State of the Union is strong due to the American people. The address discusses the nation's strength, optimism, and ability to overcome challenges. It highlights the importance of protecting freedom, liberty, and democracy. The speaker expresses confidence in America's future and its capacity to turn crises into opportunities. Specific topics mentioned include the high cost of insulin and the need to cap it, as well as actions taken regarding Russia's economy, sanctions, and releasing oil reserves in response to the war in Ukraine. The speech covers both domestic issues and international events impacting the United States."

In [49]:
pip freeze > requirements.txt

Note: you may need to restart the kernel to use updated packages.


c:\Contents\Study Materials\KaggleData\Projects\GenAI\RAG_application_using_Gemini_and_Weviatedb\venv\Lib\site-packages\IPython\utils\_process_win32.py:138: ResourceWarning: unclosed file <_io.BufferedWriter name=3>
  res = process_handler(cmd, _system_body)
c:\Contents\Study Materials\KaggleData\Projects\GenAI\RAG_application_using_Gemini_and_Weviatedb\venv\Lib\site-packages\IPython\utils\_process_win32.py:138: ResourceWarning: unclosed file <_io.BufferedReader name=4>
  res = process_handler(cmd, _system_body)
c:\Contents\Study Materials\KaggleData\Projects\GenAI\RAG_application_using_Gemini_and_Weviatedb\venv\Lib\site-packages\IPython\utils\_process_win32.py:138: ResourceWarning: unclosed file <_io.BufferedReader name=5>
  res = process_handler(cmd, _system_body)
